In [3]:
import numpy as np
import subprocess
import os
from Bio import SeqIO
from joblib import Parallel, delayed
import multiprocessing as mp
from datetime import datetime

In [22]:
# Functions

def oneHot(residue):
    mapping = dict(zip("ACDEFGHIKLMNPQRSTVWY", range(20)))
    if residue in "ACDEFGHIKLMNPQRSTVWY":
        return np.eye(20)[mapping[residue]]
    else:
        return np.zeros(20)
    
    
def reverseOneHot(encoding):
    mapping = dict(zip(range(20),"ACDEFGHIKLMNPQRSTVWY"))
    seq=''
    for i in range(len(encoding)):
        if np.max(encoding[i])>0:
            seq+=mapping[np.argmax(encoding[i])]
    return seq

def extract_seq_pdb(pdb_path):
    pdb_sequence = SeqIO.parse(open(pdb_path), 'pdb-atom')
    sequences = []
    chains = []
    total_length = 0
    for chain in pdb_sequence:
        sequences.append(str(chain.seq))
        total_length += len(str(chain.seq))
        chains.append(chain.id.replace('????:', ''))
    return sequences, chains, total_length

def run_foldx(model):
    # RepairPDB
    repair_command = "foldx --command=RepairPDB --pdb={} --ionStrength=0.05 --pH=7 --water=CRYSTAL --vdwDesign=2 --out-pdb=1 --pdbHydrogens=false".format(model)
    subprocess.run(repair_command.split(),
                        stdout=subprocess.PIPE, universal_newlines=True)
    
    # AnalyseComplex
    analyse_command = "foldx --command=AnalyseComplex --pdb=" + model.replace(".pdb", "_Repair.pdb")
    subprocess.run(analyse_command.split(),
                        stdout=subprocess.PIPE, universal_newlines=True)

def extract_foldx_energies(foldx_output_path):
    foldx_output = open(foldx_output_path, "r")
    foldx_interaction_energies = dict()
    for line in foldx_output:
        if line.startswith("./"):
            splitted_line = line.split("\t")
            group1 = splitted_line[1]
            group2 = splitted_line[2]
            interaction_energy = splitted_line[6]
            foldx_interaction_energies[group1+group2] = float(interaction_energy)    
    return foldx_interaction_energies

def run_rosetta(model):
    # Relaxation
    rosetta_relax_command = "relax.default.linuxgccrelease \
                            -fa_max_dis 9 \
                            -relax:constrain_relax_to_start_coords \
                            -ignore_unrecognized_res \
                            -missing_density_to_jump \
                            -nstruct 1 \
                            -relax:coord_constrain_sidechains \
                            -relax:cartesian \
                            -beta \
                            -score:weights beta_nov16_cart \
                            -ex1 \
                            -ex2 \
                            -relax:min_type lbfgs_armijo_nonmonotone \
                            -out:suffix _bn15_calibrated \
                            -flip_HNQ \
                            -no_optH false \
                            -s model_TCR-pMHC.pdb \
                            -out:path:pdb /home/people/idamei/rosetta/amelie_rosetta/relax_output \
                            -out:path:score /home/people/idamei/rosetta/amelie_rosetta/expeced_output"
    subprocess.run(rosetta_relax_command.split(),
                        stdout=subprocess.PIPE, universal_newlines=True)
    
    # Scoring
    rosetta_score_command = 
    subprocess.run(rosetta_score_command.split(),
                        stdout=subprocess.PIPE, universal_newlines=True)
    
def extract_rosetta_energies(rosetta_output_path)
    
def create_output(pdb_path, foldx_interaction_energies):
    # one-hot AA, M, P, TCR, foldx_MP, foldx_MA, foldx_MB, foldx_PA, foldx_PB, foldx_AB, 
    # Rosetta_total_energy, Rosetta_per_res_indiv_energies
    
    # Extract sequence from PDB
    sequences, chains, total_length = extract_seq_pdb(pdb_path)
    
    # Create output_array
    output_array = np.empty(shape=(total_length,29))
    k1 = 0
    k2 = 0
    for chain in sequences:
        chain = chains[k1]
        k1 += 1
        for aminoacid in chain:
            output_array[k2,0:20] = oneHot(aminoacid)
            if chain == "M":
                output_array[k2,20:23] = np.array([1, 0, 0])
            if chain == "P":
                output_array[k2,20:23] = np.array([0, 1, 0])
            if chain == "A" or chain == "B":
                output_array[k2,20:23] = np.array([0, 0, 1])
            output_array[k2,23:30] = list(foldx_interaction_energies.values())
            k2 += 1
    
    return output_array

def pipeline(model):
    
    try:
        # FoldX
        run_foldx(model)
    
        # Extract foldX energies
        foldx_interaction_path = "/home/ida/foldx/Interaction_" + model.replace(".pdb", "_Repair_AC.fxout")
        foldx_interaction_energies = extract_foldx_energies(foldx_interaction_path)

        # Rosetta
        run_rosetta(model)

        # Extract Rosetta energies
        rosetta_output_path = 
        extract_rosetta_energies(rosetta_output_path)

        # Create output
        model_path = "/home/ida/foldx/" + model
        output_array = create_output(model_path, foldx_interaction_energies)

    return output_array

    except error as err:
        print("Error: " err)
    

SyntaxError: invalid syntax (<ipython-input-22-d78e3da3cc73>, line 55)

In [18]:
# Calculate energy with FoldX
models = ["5men_model_TCR-pMHC.pdb", "5nmf_model_TCR-pMHC.pdb", "3tkf_model_TCR-pMHC.pdb", "5isz_model_TCR-pMHC.pdb"]

pool = mp.Pool(4)

results = []
results.append(pool.map(pipeline, [model for model in models]))

pool.close()

/home/ida/anaconda3/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3276
  warnings.warn(
/home/ida/anaconda3/lib/python3.8/site-packages/Bio/SeqIO/PdbIO.py:303: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/ida/anaconda3/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3267
  warnings.warn(
/home/ida/anaconda3/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3243
  warnings.warn(
/home/ida/anaconda3/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3259
  warnings.warn(
/home/ida/anaconda3/lib/python3.8/site-packages/Bio/SeqIO/PdbIO.py:303: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/ida/anaconda3/lib/

In [5]:
# Import input file

fastas = ["/home/ida/master-thesis/5nmf.fsa", "/home/ida/master-thesis/5nmf_mut.fsa"]
infilename = "/home/ida/master-thesis/5nmf.fsa"


In [6]:
# Model complex with TCRpMHCmodels

path_tcrpmhcmodels = "/home/ida/TCRpMHCmodels/"

path_fasta = "/home/ida/master-thesis/5nmf.fsa"

#subprocess.run(["tcrpmhc_models", "-t", path_fasta])

# Move output pdb to foldx folder

In [7]:
# Calculate energy Rosetta

# Relax
relax_command = "relax.default.linuxgccrelease -s model_TCR-pMHC.pdb -nstruct 5 -relax:default_repeats 5 -out:path:pdb /home/people/idamei/relax_output -out:path:score /home/people/idamei/expeced_output"
# Relax cartesian
relax_command = "relax.default.linuxgccrelease -s model_TCR-pMHC.pdb -nstruct 5 -relax:default_repeats 5 -out:path:pdb /home/people/idamei/relax_output -out:path:score /home/people/idamei/expeced_output -relax:dualspace true"

# Score per residue
per_res_command = "per_residue_energies.linuxgccrelease -in:file:s /home/people/idamei/relax_output/model_TCR-pMHC_0001.pdb"

# Score
score_command = "score_jd2.linuxgccrelease -in:file:s /home/people/idamei/relax_output/model_TCR-pMHC_0001.pdb"